In [14]:
#Based on the ubisense data output structure
# < > {} : \  the shit i cant find on a polish keyboard
import statistics
import csv
import numpy as np
#data_reader = csv.reader(open(r"C:/Users/nikke/Desktop/Master/masters-software/datasets/ubisense_MIRexperiment_data.txt", encoding="utf-16"), delimiter = " ")


#Maps data in  an array on the format:
#   0        1       2      3                 4             5  6  7     8     
#[index, timestamp, ID, sequence(no clue), n_meas(no clue), x, y, z, variance]
def to_array(path, delimiter):                 
    data_reader = csv.reader(open(path, encoding="utf-16"), delimiter = delimiter) #assuming utf16
    return_array=[]
    for count, x in enumerate(data_reader):
        if count > 0:
            tmp = []
            for y in x:
                #ignores empty and "ok" values
                if y != "" and y != "ok" :
                    tmp.append(y)

            return_array.append(tmp)

    return return_array

In [2]:
#example format of function call
#to_array("C:/Users/nikke/Desktop/Master/masters-software/datasets/ubisense_MIRexperiment_data.txt", " ")

In [3]:
def seperate_on_ID(array):
    return_list = []
    unique_values = []
    for sublist in array:
        if sublist[2] not in unique_values:
            unique_values.append(sublist[2])


    for ID in unique_values:
        return_list.append(list(filter(lambda c: c[2] == ID, array)))

    return return_list

In [4]:
#Returns the median and mean in an array
#Data must be at least run through to_array
def getMedianMean(data):
    #trying to find highest value of variance and stuff
    
    try:
        varArr = []
        for IDs in data:
            #srt = sorted(IDs, key=lambda x: x[8], reverse=True)
            for val in IDs:
                varArr.append(float(val[8]))

        #print(sorted(varArr, reverse=True))
        return[statistics.median(varArr), statistics.mean(varArr)]
    
    #Catch indexerror of using an unsplit array
    except IndexError:
        varArr = []
        
        #srt = sorted(IDs, key=lambda x: x[8], reverse=True)
        for val in data:
            varArr.append(float(val[8]))

        #print(sorted(varArr, reverse=True))
        return[statistics.median(varArr), statistics.mean(varArr)]
    


In [17]:
#Reduces list to indices supplied. Should probably be
#assumes list is seperated on ID, and I guess you better know in which order those IDs are.
def readyData(data, indices, variance):
    try:
        #List comprehension is incomprehensible 
        return [[[float(val[index]) for index in indices] for val in inner if float(val[8]) < variance] for inner in data]
        
    except IndexError:
        print("Index out of bounds - Use seperate_on_ID function or think harder")

In [6]:
#quickdata = seperate_on_ID(to_array("C:/Users/nikke/Desktop/Master/masters-software/datasets/ubisense_MIRexperiment_data.txt", " "))

#test = readyData(quickdata, [5,6], 4)

#print(test[0])

In [7]:
def create_input_output_arrays(xy_pairs, input_seq_len, output_seq_len):
    """
    Creates input and output arrays from a 2D array of x-y pairs.

    Args:
        xy_pairs (list of lists): A 2D array of x-y coordinate pairs.
        input_seq_len (int): The length of the input sequence.
        output_seq_len (int): The length of the output sequence.

    Returns:
        array: An array of the input and output arrays.

    """

    # Initialize empty lists for input and output sequences
    input_seq = []
    output_seq = []

    # Loop through xy_pairs, starting at index input_seq_len
    for i in range(input_seq_len, len(xy_pairs) - output_seq_len + 1):

        # Get the input sequence by selecting the previous input_seq_len xy pairs
        input_seq.append(xy_pairs[i - input_seq_len:i])

        # Get the output sequence by selecting the next output_seq_len xy pairs
        output_seq.append(xy_pairs[i:i + output_seq_len])

    # Convert the input and output sequences to numpy arrays
    input_seq = np.array(input_seq)
    output_seq = np.array(output_seq)

    # Return the input and output arrays as a tuple
    return [input_seq, output_seq]

In [16]:
#quickdata = seperate_on_ID(to_array("C:/Users/nikke/Desktop/Master/masters-software/datasets/ubisense_MIRexperiment_data.txt", " "))

#test = readyData(quickdata, [5,6], 4)
#create_input_output_arrays(test[0], 3, 2)

In [18]:
def create_input_output_pairs(xy_pairs, input_seq_len, output_seq_len):
    X, Y = [], []
    for i in range(len(xy_pairs) - input_seq_len - output_seq_len + 1):
        input_seq = [[float(val) for val in xy_pairs[j+k]] for k in range(input_seq_len) for j in range(i)]
        output_seq = [[float(val) for val in xy_pairs[j+k]] for k in range(output_seq_len) for j in range(i+input_seq_len)]
        X.append(input_seq)
        Y.append(output_seq)
    return np.array(X), np.array(Y)